# Running Native Functions


Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

- Writing logic around how to run a prompt that changes the prompt's outcome.
- Using external data sources to gather data to concatenate into your prompt.
- Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


In [ ]:
!python -m pip install semantic-kernel==0.9.6b1

In [2]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [3]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.utils.settings import azure_openai_settings_from_dot_env, openai_settings_from_dot_env

kernel = Kernel()

if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name="gpt-35-turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.


First, let's create our native function.


In [4]:
import random

from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use.


In [5]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long. It must be this length.
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="story",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

generate_number_plugin = kernel.add_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")

In [6]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]
number_result = await generate_number_three_or_higher(kernel, input=6)
print(number_result)

3


In [7]:
story = await corgi_story.invoke(kernel, input=number_result.value)

_Note: depending on which model you're using, it may not respond with the proper number of paragraphs._


In [8]:
print(f"Generating a corgi story exactly {number_result.value} paragraphs long.")
print("=====================================================")
print(story)

Generating a corgi story exactly 3 paragraphs long.
Once upon a time in a quaint little town, two corgis named Max and Ruby set out on an exciting adventure. Their tails wagging with excitement, they trotted through the streets, greeting everyone they met with a friendly bark and a wag of their fluffy tails. As they made their way through the town, they encountered all sorts of new and interesting things, from colorful flowers in the park to friendly ducks at the pond.

As they explored, Max and Ruby realized that the world was full of diversity and beauty. They met dogs of all shapes and sizes, as well as people from different walks of life. They learned that everyone, no matter how different, had something special to offer. With each new encounter, they made new friends and spread joy wherever they went.

At the end of the day, Max and Ruby returned home, tired but happy. They had learned that the world is a wonderful place, full of endless possibilities and new experiences. They rea

## Kernel Functions with Annotated Parameters

That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the Python's `Annotated` class to hold these variables.


In [9]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = Kernel()

if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's start with the native function. Notice that we're add the `@kernel_function` decorator that holds the name of the function as well as an optional description. The input parameters are configured as part of the function's signature, and we use the `Annotated` type to specify the required input arguments.


In [11]:
import random
import sys

from semantic_kernel.functions import kernel_function

if sys.version_info >= (3, 9):
    from typing import Annotated
else:
    from typing_extensions import Annotated


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @kernel_function(
        name="GenerateNumber",
        description="Generate a random number between min and max",
    )
    def generate_number(
        self,
        min: Annotated[int, "the minimum number of paragraphs"],
        max: Annotated[int, "the maximum number of paragraphs"] = 10,
    ) -> Annotated[int, "the output is a number"]:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(min, max))
        except ValueError as e:
            print(f"Invalid input {min} and {max}")
            raise e

In [12]:
generate_number_plugin = kernel.add_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")
generate_number = generate_number_plugin["GenerateNumber"]

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.


In [13]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

Let's generate a paragraph count.


In [14]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value
print(f"Generating a corgi story {num_paragraphs} paragraphs long.")

Generating a corgi story 5 paragraphs long.


We can now invoke our corgi_story function using the `kernel` and the keyword arguments `paragraph_count` and `language`.


In [15]:
# Pass the output to the semantic story function
desired_language = "Spanish"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [16]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 5 paragraphs long in Spanish.
Había una vez dos adorables Corgis llamados Coco y Luna. Eran los mejores amigos y les encantaba ir de aventuras juntos. Un día, decidieron explorar el bosque cercano a su casa.

Mientras caminaban por el bosque, Coco y Luna descubrieron un hermoso arroyo. Se miraron el uno al otro con emoción y decidieron seguirlo. Pronto, se encontraron con una familia de patos nadando en el arroyo. Los Corgis se detuvieron a observar y se dieron cuenta de lo pacíficos y felices que parecían los patos juntos.

Coco y Luna aprendieron una valiosa lección ese día: la importancia de la amistad y la armonía. Se dieron cuenta de que, al igual que los patos, ellos también eran más felices cuando estaban juntos y se apoyaban mutuamente. Decidieron que siempre estarían allí el uno para el otro, pase lo que pase.

La aventura de Coco y Luna en el bosque fue una experiencia maravillosa que fortaleció su amistad. A partir de ese día, se comprometieron a seg

## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Prompt Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md).


In [17]:
import random

from semantic_kernel.functions import kernel_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @kernel_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @kernel_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [18]:
generate_names_plugin = kernel.add_plugin(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [19]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [20]:
if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="corgi-new",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.add_function(
    function_name="CorgiStoryUpdated",
    plugin_name="CorgiPluginUpdated",
    prompt_template_config=prompt_template_config,
)

In [21]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value

In [22]:
desired_language = "French"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [23]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 4 paragraphs long in French.
Il était une fois deux corgis, Hamilton et Tuna, qui étaient les meilleurs amis du monde. Un jour, ils ont décidé de partir à l'aventure dans la grande forêt voisine. Ils étaient excités et prêts à affronter tout ce qui se présentait à eux.

En chemin, ils ont rencontré toutes sortes de créatures de la forêt, certaines effrayantes et d'autres amicales. Mais Hamilton et Tuna ont gardé leur courage et leur bonne humeur tout au long de leur voyage. Ils ont partagé des moments de joie et de peur, mais leur amitié les a toujours aidés à surmonter les obstacles.

Finalement, après une journée pleine d'aventures, Hamilton et Tuna sont rentrés chez eux, épuisés mais heureux. Leur grande aventure les avait rapprochés encore plus et leur avait appris qu'avec de la persévérance et de l'amitié, on peut surmonter n'importe quel défi. Ils ont réalisé que l'important n'était pas tant l'endroit où l'on va, mais avec qui on le partage.

Depuis ce jo

### Recap

A quick review of what we've learned here:

- We've learned how to create native and prompt functions and register them to the kernel
- We've seen how we can use Kernel Arguments to pass in more custom variables into our prompt
- We've seen how we can call native functions within a prompt.
